In [2]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser 
from langchain.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")

In [18]:
model.invoke("Bist du nicht chat gpt 3?")
# returns JSON data with model output and metadata 

AIMessage(content='Ja, ich bin ein Modell von OpenAI und basiere auf der Technologie GPT-3. Ich bin ein KI-gesteuerter Chatbot.', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 17, 'total_tokens': 51}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6d14c275-5ae8-47aa-b098-af97da169ef9-0')

In [5]:
response = model.invoke("Erkläre LLMs in drei Sätzen")
response.content

'LLMs, kurz für "Large Language Models", sind fortgeschrittene KI-Systeme, die darauf trainiert sind, menschliche Sprache zu verstehen und zu generieren. Sie basieren auf einer Architektur namens Transformer, die große Mengen an Textdaten verarbeitet, um Muster, Kontext und Sprachnuancen zu lernen. Diese Modelle werden in vielfältigen Anwendungen genutzt, von der Textgenerierung und Übersetzung bis hin zur Beantwortung von Fragen und der Unterstützung bei der Textbearbeitung, indem sie fähig sind, kohärente und kontextbezogene Antworten zu generieren.'

In [6]:
parser = StrOutputParser() # Initialisierung eines Objekts zum Extrahieren des Strings 
chain = model | parser # Output des Model wird in parser Objekt gepiped
chain.invoke("Erkläre LLMs in drei Sätzen")

'LLMs, oder "Large Language Models", sind fortschrittliche KI-Systeme, die darauf trainiert sind, menschliche Sprache zu verstehen und zu generieren. Sie basieren auf neuronalen Netzwerken, insbesondere auf Architekturen wie Transformer, die große Mengen an Textdaten verarbeiten, um Muster und Kontexte zu lernen. LLMs werden in verschiedenen Anwendungen verwendet, darunter Textgenerierung, Übersetzung, Zusammenfassung und andere sprachbasierte Aufgaben, um Benutzern zu helfen, Informationen effizienter zu verarbeiten und zu verstehen.'

In [7]:
template = """
Beantworte die Frage basierend auf dem unten gegebenen Kontext. 
Wenn das Beantworten der Frage nicht möglich ist durch den gegebenen Kontekt, antworte "Ich weiß es nicht". 

Kontext: {context}

Frage: {question}
"""

prompt = ChatPromptTemplate.from_template(template) # Template Prompt Objekt erzeugen


In [8]:
model.invoke(prompt.format(context="Es gibt auschließlich weiße Schwäne.", question = "Gibt es schwarze Schwäne?")).content


'Ich weiß es nicht.'

Du heißt Max.


In [9]:
chain = prompt | model | parser
chain.invoke({
    "context": "Es gibt ausschließlich weiße Schwäne.", 
    "question": "Gibt es schwarze Schwäne?"
})

'Ich weiß es nicht.'

In [10]:
from operator import itemgetter

diff_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

diff_chain = (
    # dic um Input zu definieren, dies wird dann in diff_prompt gepiped
    {"answer": chain, # Antwort wird aus der anderen Chain als Input genommen
     "language": itemgetter("language")} # Sprache aus Dictonary
    | diff_prompt | model | parser 
)

diff_chain.invoke(
    {
        "context": "Alle Schwäne sind weiß, außer in Australien. Dort gibt es schwarze.",
        "question": "Gibt es schwarze Schwäne?", 
        "language": "Italienisch"
    }
)

'Das Wort "Ja" auf Italienisch ist "Sì".'

In [4]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Gibt es schwarze Schwäne?")

print(f"Embedding length: {len(embedded_query)}") # Dimension der Embedding Liste

Embedding length: 1536
[0.00901121263534296, -0.03669295798076368, -0.00873078020641617, -0.025039455366582045, -0.01990443465229751, 0.006879930087054519, -0.0007376917735520665, -0.012351468412199479, -0.009098458010849193, -0.04162856202435205]


In [6]:
sen1 = embeddings.embed_query("Es gibt in Europa nur weiße Schwäne")
sen2 = embeddings.embed_query("Es gibt schwarze Schwäne in Australien")

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

query_sen1_sim = cosine_similarity([embedded_query], [sen1])[0][0]
query_sen2_sim = cosine_similarity([embedded_query], [sen2])[0][0]
query_sen1_sim, query_sen2_sim

(0.8740940461805541, 0.9229002354973087)

In [20]:
import rag
run_id= "25eec295-c3d5-4946-80bf-6e17f19e15af"
client = rag.get_client()
feedback_record = client.create_feedback(
    run_id,
    key = "test",
    score=1,
                )

In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langsmith import Client
from langchain.callbacks import LangChainTracer

# load .env Variablen 
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# tracing mit Langsmith from Langchain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG-project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
client = Client()

In [2]:
def send_feedback(run_id,score):
    key =f"feedback_{run_id}"
    client.create_feedback(
        run_id,
        key=key,
        score=score,
        )

In [4]:
send_feedback("217b483c-e520-498d-bb49-d0e0b5a43ff0",1)

In [86]:
list = [True, False]
sum(list)
print("ssdjkwjd"+"ksdaaö")

ssdjkwjd


ksdaaö


In [126]:
from atlassian import Confluence
import os
from dotenv import load_dotenv
username = 'max1912sch@gmail.com'        # Your Confluence username       # Your Confluence API token
space_key = 'WD'                     # The key for your Confluence space
url="https://rag-project.atlassian.net/"
load_dotenv()
CONFLUENCE_API_KEY=os.getenv("CONFLUENCE_API_KEY")
# Configuration

# Initialize Confluence
confluence = Confluence(
    url=url,
    username=username,
    password=api_key
)


In [88]:
# loop um mehr als nur 100 spaces zu bekommen
def get_all_pages(confluence, space):
    start = 0
    limit = 100
    _all_pages = []
    while True:
        pages = confluence.get_all_pages_from_space(space, start, limit, status=None, expand="body.storage", content_type='page')
        _all_pages = _all_pages + pages
        if len(pages) < limit: 
            break
        start = start + limit
    return _all_pages

In [90]:
pages = get_all_pages(confluence, "WD")
pages

[{'id': '33258',
  'type': 'page',
  'status': 'current',
  'title': 'Wissensdatenbank',
  'macroRenderedOutput': {},
  'body': {'storage': {'value': '<ac:layout><ac:layout-section ac:type="fixed-width" ac:breakout-mode="default"><ac:layout-cell><ac:structured-macro ac:name="panel" ac:schema-version="1" ac:macro-id="8680523d-e572-45c4-8e6c-c333393513dd"><ac:parameter ac:name="bgColor">#EAE6FF</ac:parameter><ac:rich-text-body><h3><strong>Willkommen in Ihrem neuen Wissensdatenbank-Bereich!</strong></h3><ul><li><p>Wir haben einige Vorschl&auml;ge und Platzhalter hinzugef&uuml;gt. Alles ist anpassbar.</p></li><li><p>Erste Schritte mit Seitenvorlagen:</p><ul><li><ac:link ac:card-appearance="inline"><ri:page ri:content-title="Vorlage&nbsp;&ndash; Anleitung" ri:version-at-save="1" /><ac:link-body>Vorlage&nbsp;&ndash; Anleitung</ac:link-body></ac:link></li><li><ac:link ac:card-appearance="inline"><ri:page ri:content-title="Vorlage&nbsp;&ndash; Fehlerbehebungsartikel" ri:version-at-save="1" /><

In [95]:
bodies =[]
for page in pages:
    bodies.append(page["body"]["storage"]["value"])
    
bodies
    
titles = []
for page in pages: 
    titles.append(page["title"])    
titles    

['Wissensdatenbank', 'Eden Hazard', 'Arjen Robben', 'Lionel Messi']

In [100]:
from bs4 import BeautifulSoup

pages = get_all_pages(confluence, "WD")
bodies = ""
for page in pages:
    bodies = bodies + "\n\n\n##############################################################\n\n\n"
    bodies = bodies + page["title"] +  ":\n\n" + page["body"]["storage"]["value"]

bodies

soup = BeautifulSoup(bodies, 'html.parser')


#     # Extract text from the parsed HTML
text = soup.get_text(separator="")

text

'\n\n\n##############################################################\n\n\nWissensdatenbank:\n\n#EAE6FFWillkommen in Ihrem neuen Wissensdatenbank-Bereich!Wir haben einige Vorschläge und Platzhalter hinzugefügt. Alles ist anpassbar.Erste Schritte mit Seitenvorlagen:Vorlage\xa0– AnleitungVorlage\xa0– FehlerbehebungsartikelWillkommen! Wie können wir Ihnen helfen?largeHilfeartikel durchsuchenInfoLegen Sie kurz dar, welche Aufgaben Sie oder Ihr Team haben, welche Informationen in diesem Bereich zu finden sind und was sonst noch wissenswert ist.KontaktTeam@email.com#slackchannelTicket erstellen: <link>Top-Themen🍓 Thema\xa01Link zur SeiteLink zur SeiteLink zur Seite🥝 Thema\xa02Link zur SeiteLink zur SeiteLink zur Seite🍍 Thema\xa03Link zur SeiteLink zur SeiteLink zur Seite🍐 Thema\xa04Link zur SeiteLink zur SeiteLink zur Seite🍋 Thema\xa05Link zur SeiteLink zur SeiteLink zur Seite🍒 Thema\xa06Link zur SeiteLink zur SeiteLink zur SeiteTeamnameSupportLink zur SeiteLink zur SeiteLink zur SeiteZugehö

In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain.docstore.document import Document

doc =  Document(page_content=text)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
knowledge_chunks = text_splitter.split_documents([doc])
embeddings = OpenAIEmbeddings()
knowledge_chunks


[Document(page_content='##############################################################\n\n\nWissensdatenbank:'),
 Document(page_content='Wissensdatenbank:\n\n#EAE6FFWillkommen in Ihrem neuen Wissensdatenbank-Bereich!Wir haben einige Vorschläge und Platzhalter hinzugefügt. Alles ist anpassbar.Erste Schritte mit Seitenvorlagen:Vorlage\xa0– AnleitungVorlage\xa0– FehlerbehebungsartikelWillkommen! Wie können wir Ihnen helfen?largeHilfeartikel durchsuchenInfoLegen Sie kurz dar, welche Aufgaben Sie oder Ihr Team haben, welche Informationen in diesem Bereich zu finden sind und was sonst noch wissenswert ist.KontaktTeam@email.com#slackchannelTicket erstellen: <link>Top-Themen🍓 Thema\xa01Link zur SeiteLink zur SeiteLink zur Seite🥝 Thema\xa02Link zur SeiteLink zur SeiteLink zur Seite🍍 Thema\xa03Link zur SeiteLink zur SeiteLink zur Seite🍐 Thema\xa04Link zur SeiteLink zur SeiteLink zur Seite🍋 Thema\xa05Link zur SeiteLink zur SeiteLink zur Seite🍒 Thema\xa06Link zur SeiteLink zur SeiteLink zur SeiteT

In [107]:
pinecone = PineconeVectorStore.from_documents(
    knowledge_chunks, embeddings, index_name="llm-km")

In [108]:
pinecone.similarity_search(question)[:3]

[Document(page_content='Er debütierte für Real am 18. September beim 2:1 im Champions-League-Spiel gegen Werder Bremen, als er nach 84 Minuten für Raúl auf den Platz kam. Er wurde schnell fester Bestandteil der Mannschaft und trug mit vier Toren in 21 Ligaspielen zur spanischen Meisterschaft 2008 bei.Zu Beginn der neuen Saison gewann er mit Real auch den spanischen Supercup in zwei Spielen gegen den Pokalsieger Valencia CF. In der Liga reichte es nurmehr zur Vizemeisterschaft. Wesentlich wurde aber für ihn der Wechsel von Trainer Bernd Schuster zu Juande Ramos ab dem 15. Spieltag. Ramos versetzte den linksfüßigen Linksaußen auf die rechte Seite. Dort wurde der Sprint an der Außenlinie zur Torlinie, von dort schräg zurück in den Strafraum und Schuss auf das Tor mit Links zu seinem Markenzeichen. Nachdem er bislang in jener Saison nur einmal getroffen hatte, erzielte er auf Rechts sechs weitere Treffer. Insgesamt kam er 2008/09 auf 29 Ligaspiele.Zur Saison 2009/10 war mit Florentino Pére

In [127]:
pages = get_all_pages(confluence, space_key)
bodies = ""
for page in pages:
    bodies = bodies + "\n\n\n#################################\n\n\n"
    bodies = bodies + page["title"] +  ":\n\n" + page["body"]["storage"]["value"]
        
# html Zeug entfernen    
soup = BeautifulSoup(bodies, 'html.parser')

# Extract text from the parsed HTML
text = soup.get_text(separator="")
        
# Saving the text to a .txt file
file_path = "confluence_data.txt"  # Define the path and file name
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(text)


In [122]:
import os
from dotenv import load_dotenv
from atlassian import Confluence
from bs4 import BeautifulSoup


# Configuration
username = 'max1912sch@gmail.com'        # Your Confluence username       # Your Confluence API token
space_key = 'WD'                     # The key for your Confluence space
url="https://rag-project.atlassian.net/"
load_dotenv()
BRO=os.getenv("CONFLUENCE_API")
# Initialize Confluence

print(BRO)
   

In [135]:
from langchain_community.document_loaders import TextLoader
knowledge = TextLoader("confluence_data.txt")

knowledge_base = knowledge.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
knowledge_chunks = text_splitter.split_documents(knowledge_base)
embeddings = OpenAIEmbeddings()
pinecone = PineconeVectorStore.from_documents(
    knowledge_chunks, embeddings, index_name="llm-km")

In [6]:
import os
from dotenv import load_dotenv
from atlassian import Confluence
from bs4 import BeautifulSoup

# Configuration
load_dotenv()
username = os.getenv("confluence_username")
url=os.getenv("confluence_url")
CONFLUENCE_API_KEY=os.getenv("CONFLUENCE_API")
# Initialize Confluence
space_key = os.getenv("confluence_spacekey")
confluence = Confluence(
    url=url,
    username=username,
    password=CONFLUENCE_API_KEY
    )

# loop um mehr als nur 100 pages auf einmal zu bekommen
def get_all_pages(confluence, space):
    start = 0
    limit = 100
    all_pages = []
    while True:
        pages = confluence.get_all_pages_from_space(space, start, limit, status=None, expand="body.storage", content_type='page')
        all_pages = all_pages + pages
        if len(pages) < limit: 
            break
        start = start + limit
    return all_pages

get_all_pages(confluence,space_key)    

In [5]:
def get_data_confluence():
    pages = get_all_pages(confluence, space_key)
    bodies = ""
    for page in pages:
        bodies = bodies + "\n\n\n#################################\n\n\n"
        bodies = bodies + page["title"] +  ":\n\n" + page["body"]["storage"]["value"]
        
    # html Zeug entfernen    
    soup = BeautifulSoup(bodies, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text(separator="")
        
    # Saving the text to a .txt file
    file_path = "confluence_data.txt"  # Define the path and file name
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)
get_data_confluence()

TypeError: 'NoneType' object is not iterable